In [ ]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal


class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return str(o)
        return super(DecimalEncoder, self).default(o)


dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
table = dynamodb.Table('A0195028B')
FD001 = {}
FD002 = {}
response = table.scan()
for n in range(1, 101):
    id_name = "FD001_" + str(n)
    FD001[id_name] = {}
    for j in response['Items'][0].keys():
        FD001[id_name][j] = []
    FD001[id_name].pop('Matric No')
    FD001[id_name].pop('id')
    FD001[id_name].pop('timestamp')
    FD001[id_name].pop('clientToken')
for n in range(1, 261):
    id_name = "FD002_" + str(n)
    FD002[id_name] = {}
    for j in response['Items'][0].keys():
        FD002[id_name][j] = []
    FD002[id_name].pop('Matric No')
    FD002[id_name].pop('id')
    FD002[id_name].pop('timestamp')
    FD002[id_name].pop('clientToken')
for i in response['Items']:
    if i['id'][4] == '1':
        for k in FD001[i['id']].keys():
            FD001[i['id']][k].append(float(i[k]))
    else:
        for k in FD002[i['id']].keys():
            FD002[i['id']][k].append(float(i[k]))

while 'LastEvaluatedKey' in response:
    response = table.scan(
        ExclusiveStartKey=response['LastEvaluatedKey']
    )
    for i in response['Items']:
        if i['id'][4] == '1':
            for k in FD001[i['id']].keys():
                FD001[i['id']][k].append(float(i[k]))
        else:
            for k in FD002[i['id']].keys():
                FD002[i['id']][k].append(float(i[k]))
print(FD001)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
life_1 = []
life_2 = []
for i in range(1, 101):
    id_name = "FD001_" + str(i)
    life_1.append(len(FD001[id_name]['circle']))
for i in range(1, 261):
    id_name = "FD002_" + str(i)
    life_2.append(len(FD002[id_name]['circle']))
# life_2.sort()
# print(life_2)
# life_1.sort()
# print(life_1)
mean_1 = np.mean(life_1)
mean_2 = np.mean(life_2)
plt.figure(1,figsize=(15,6), dpi=240, facecolor="white")
plt.subplot(1,2,1)
plt.title("The useful life of FD001")
plt.hist(life_1, bins=10, range=(125, 375), density=0, facecolor="deepskyblue",edgecolor="black")
plt.text(mean_1+10, 25, "average useful life: "+str(round(mean_1, 2)), size = 15)
plt.plot([mean_1,mean_1],[0,30],color="darkorange",linestyle="--")
plt.subplot(1,2,2)
plt.title("The useful life of FD002")
plt.hist(life_2, bins=11, range=(125, 400), density=0, facecolor="deepskyblue",edgecolor="black")
plt.plot([mean_2,mean_1],[0,70],color="darkorange",linestyle="--")
plt.text(mean_2+10, 60, "average useful life: "+str(round(mean_2, 2)), size = 15)
plt.show()

In [ ]:
import pandas as pd
FD1 = pd.DataFrame()
for i in range(1,101):
    name_id = "FD001_" + str(i)
    dFD = pd.DataFrame(FD001[name_id])
    FD1 = FD1.append(dFD)
order = ['circle', 'os1', 'os2', 'os3'] + ['sensor' + str(i) for i in range(1, 22)]
FD1 = FD1[order]
FD1.describe().transpose()
FD1.drop(['circle', 'sensor1', 'sensor5', 'sensor6', 'sensor10', 'sensor16', 'sensor18', 'sensor19', 'os3'], 
         axis=1, inplace=True)
FD2 = pd.DataFrame()
for i in range(1,261):
    name_id = "FD002_" + str(i)
    dFD = pd.DataFrame(FD002[name_id])
    FD2 = FD2.append(dFD)
order = ['circle', 'os1', 'os2', 'os3'] + ['sensor' + str(i) for i in range(1, 22)]
FD2 = FD2[order]

In [ ]:
plt.figure(dpi=240)
FD2.hist(bins=50, figsize=(18,16),facecolor="deepskyblue")
plt.show()

In [ ]:
plt.figure(dpi=240)
FD1.hist(bins=50, figsize=(18,16),facecolor="deepskyblue")
plt.show()

In [ ]:
FD001_1 = FD001['FD001_1'].copy()
t = FD001_1['circle']
d = ['circle','sensor1','os3','sensor5','sensor6','sensor10','sensor16','sensor18','sensor19']
[FD001_1.pop(i) for i in d]
j = 1
plt.figure(figsize=(10,20), dpi=240)
for i in FD001_1.keys():
    plt.subplot(8,2,j)
    plt.plot(t, FD001_1[i], color='deepskyblue')
    plt.xlabel("time")
    plt.ylabel(i)
    j += 1
plt.subplots_adjust(wspace =0.3, hspace =0.3)
plt.show()

In [ ]:
import pywt
def wavelet_denoising(data):
    db4 = pywt.Wavelet('db4')
    if type(data) is not None:
        coeffs = pywt.wavedec(data, db4)
        coeffs[-1] *= 0
        coeffs[-2] *= 0
        coeffs[-3] *= 0
        meta = pywt.waverec(coeffs, db4)
        return meta
plt.figure(figsize=(10,20), dpi=240)
j = 1
for i in FD001_1.keys():
    plt.subplot(8,2,j)
    w = wavelet_denoising(FD001_1[i])
    plt.plot(t, w, color='deepskyblue')
    plt.xlabel("time")
    plt.ylabel(i)
    j += 1
plt.subplots_adjust(wspace =0.3, hspace =0.3)
plt.show()